# CQF Exam One

## Optimal Portfolio Allocation

Define $\Sigma$ and vector 1

In [ ]:
import numpy as np


# Define S
S = np.array([  [0.07, 0, 0, 0],
                [0, 0.28, 0, 0],
                [0, 0, 0.25, 0],
                [0, 0, 0, 0.31]])

# Define R
R = np.array([  [1, 0.4, 0.3, 0.3],
                [0.4, 1, 0.27, 0.42],
                [0.3, 0.27, 1, 0.5],
                [0.3, 0.42, 0.5, 1]])


# Compute the matrix multiplication
Sigma = np.dot(np.dot(S, R), S)

# Calculate the inverse of Sigma
Sigma_inv = np.linalg.inv(Sigma)

# Define the vector 1
vector_1 = np.ones((4, 1))


Check if $\Sigma$ is positive definite

In [ ]:
eigenvalues, _ = np.linalg.eig(Sigma)
print([eigenvalue.round(4) for eigenvalue in eigenvalues])

Compute $\lambda$

In [ ]:
# Perform the necessary multiplications
result = np.dot(np.dot(Sigma_inv, vector_1).T, vector_1)

# Calculate lambda
lambda_value = 1 / result[0, 0]

print("Lambda =", lambda_value)

Solve for $w^*$

In [ ]:
w_star =  np.dot(lambda_value, np.dot(Sigma_inv,vector_1))
print(w_star)

---

### Question 2

In [ ]:
# Define mean return vector mu
mu = np.array([[0.05], [0.07], [0.15], [0.22]])
# Define constrain m = 7%
m = 0.07

Compute the new correlation matrix $\Sigma_{1}$, $\Sigma_{2}$ and $\Sigma_{3}$

In [ ]:
# Scaling factors
scaling_factors = [1, 1.3, 1.8]

# Apply scaling and constraints using map() function
new_corr_matrices = list(map(lambda factor: np.clip(R * factor, None, 0.99), scaling_factors))

# Set diagonal elements back to 1 for each new correlation matrix
for matrix in new_corr_matrices:
    np.fill_diagonal(matrix, 1)

# Compute new sigma

Sigma_1 = Sigma
Sigma_2 = np.dot(np.dot(S, new_corr_matrices[1]), S)
Sigma_3 = np.dot(np.dot(S, new_corr_matrices[2]), S)

Build a optimization function to compute vector $w^*$ given the input $m=7\%$, correlation matrix $\Sigma$ and mean return $\mu$

In [ ]:
def optimize(Sigma, mu, m):
    vector_1 = np.ones((4, 1))
    
    # Compute A, B, and C
    A = np.dot(np.dot(vector_1.T, Sigma_inv), vector_1)
    B = np.dot(np.dot(vector_1.T, Sigma_inv), mu)
    C = np.dot(np.dot(mu.T, Sigma_inv), mu)
    
    # Compute lambda and gamma
    denominator = A*C - B**2
    lambda_val = (A*m - B) / denominator
    gamma = (C - B*m) / denominator
    
    # Compute w_star
    w_star = np.dot(Sigma_inv, (lambda_val * mu + gamma * vector_1))
    
    return w_star


Compute the optimize vector $w_1^*$, $w_2^*$ and $w_3^*$

In [ ]:
w_1_star = optimize(Sigma_1, mu, m)
w_2_star = optimize(Sigma_2, mu, m)
w_3_star = optimize(Sigma_3, mu, m)

print("w1*")
print(w_1_star)
print()
print("w2*")
print(w_2_star)
print()
print("w3*")
print(w_3_star)

Compute portfolio risks $\sigma_{\Pi} = \sqrt{w^{\top}\Sigma w}$

In [ ]:
vol_1 = np.sqrt(np.dot(np.dot(w_1_star.T,Sigma_1), w_1_star))
vol_2 = np.sqrt(np.dot(np.dot(w_2_star.T,Sigma_2), w_2_star))
vol_3 = np.sqrt(np.dot(np.dot(w_3_star.T,Sigma_3), w_3_star))

print("Volatily for portfolio 1 ", vol_1)
print("Volatily for portfolio 2 ", vol_2)
print("Volatily for portfolio 3 ", vol_3)

---

## Understanding Risk

### Question 3

Compute loss probability given SR annualy

In [31]:
import scipy.stats as stats

# Given annualized Sharpe Ratio
SR_annual = 0.53

# Calculate daily Sharpe Ratio
SR_daily = SR_annual / np.sqrt(252)

# Calculate loss probabilities using CDF of standard normal distribution
loss_prob_daily = stats.norm.cdf(-SR_daily)
loss_prob_monthly = stats.norm.cdf(-SR_annual / np.sqrt(12))
loss_prob_quarterly = stats.norm.cdf(-SR_annual / np.sqrt(4))
loss_prob_annual = stats.norm.cdf(-SR_annual)

# Print the results
print("Loss Probability (Daily):", loss_prob_daily)
print("Loss Probability (Monthly):", loss_prob_monthly)
print("Loss Probability (Quarterly):", loss_prob_quarterly)
print("Loss Probability (Annual):", loss_prob_annual)

Loss Probability (Daily): 0.4866830433008509
Loss Probability (Monthly): 0.439199996693031
Loss Probability (Quarterly): 0.395504730907446
Loss Probability (Annual): 0.29805596539487644


---

### Question 4

Generate above 700 random allocation

In [ ]:
import pandas as pd

# Given data
returns = mu 
volatility = np.array([[0.07], [0.28], [0.25], [0.31]])

# Number of random portfolios to generate
num_portfolios = 700

# Initialize arrays to store portfolio statistics
portfolio_rets = np.zeros(num_portfolios)
portfolio_vols = np.zeros(num_portfolios)
sharpe_ratios = np.zeros(num_portfolios)

# Generate random portfolio allocations and compute statistics
for i in range(num_portfolios):
    # Generate random weights
    weights = np.random.random(4)
    # Normalize to ensure sum of weights is 1
    weights /= np.sum(weights)

    # Compute portfolio mean and variance
    portfolio_ret = np.dot(weights, returns)
    portfolio_vol = np.sqrt(np.dot(weights, np.dot(Sigma, weights.T)))

    # Calculate Sharpe ratio
    sharpe_ratio = portfolio_ret[0] / portfolio_vol

    # Store results
    portfolio_rets[i] = portfolio_ret[0]
    portfolio_vols[i] = portfolio_vol
    sharpe_ratios[i] = sharpe_ratio

# Create a DataFrame to store portfolio returns, volatilities, and Sharpe ratios
portfolio_df = pd.DataFrame({
    "Returns": portfolio_rets,
    "Volatilities": portfolio_vols,
    "Sharpe Ratios": sharpe_ratios
})

# Results
portfolio_df.head()


In [ ]:
# Import plotly express for EF plot
import plotly.express as px

# Find the index of the portfolio with the maximum Sharpe ratio
max_sharpe_index = np.argmax(sharpe_ratios)

# Plot the data using Plotly Express
fig = px.scatter(portfolio_df, x="Volatilities", y="Returns", color="Sharpe Ratios",
                 title="Monte Carlo Simulated Portfolio",
                 labels={"Portfolio Volatilities": "Volatility",
                         "Portfolio Returns": "Return"},
                 width=800, height=500).update_traces(mode='markers', marker=dict(symbol='cross'))

# Add the portfolio with the maximum Sharpe ratio to the plot with a star symbol
fig.add_scatter(
        mode='markers',
        x=[portfolio_df.iloc[max_sharpe_index]['Volatilities']],
        y=[portfolio_df.iloc[max_sharpe_index]['Returns']],
        marker=dict(symbol='star', size=10, color='red'),
        name='Max Sharpe').update(layout_showlegend=False)

fig.show()

---

### Question 5

VaR formula

$$
\text{VaR}_{10D,t} =  \text{Factor} \times \sigma_{t} \times \sqrt{10}
$$

Find `Factor` using inverse cdf function

In [ ]:
from scipy.stats import norm
# Compute `factor`
factor = norm.ppf(0.01)
factor

Import the nasdaq100 data and calulate returns and VaR given the above equation

In [ ]:
# Compute returns
df = pd.read_csv("../exam/nasdaq100.csv",parse_dates=["Date"], index_col="Date")
df['rets'] = np.log(df['Closing Price']).diff()

# Compute 10 days VaR 
window = 21
df['VaR_10D'] = factor * df['rets'].rolling(window=window).std() * np.sqrt(10)

# Compute 10 days forward returns
df['rets_10D'] = np.log(df['Closing Price'].shift(-11) / df['Closing Price'].shift(-1))

# Identify breaches
df['breach'] = (df['rets_10D'] < df['VaR_10D']).astype(int)

df.head()

1. The count and percentage of VaR breaches.

In [ ]:
mask = df['breach'] == 1
pct_var_breaches = df[mask]['breach'].count() / len(df)
print('No. of Breaches', df[mask]['breach'].count())
print('Pecentage VaR Breaches', pct_var_breaches)

2. The count of consecutive VaR breaches. (1, 1, 1 indicates two consecutive occurrences)

In [ ]:
# Convert the breach column to a list for easier manipulation
breach_list = df['breach'].tolist()

# Initialize variables
consecutive_counts_list = []

# Iterate through the list of breach values
for i in range(len(breach_list) - 1):
    if breach_list[i] == 1 and breach_list[i + 1] == 1:
        consecutive_counts_list.append(1)
    else:
        consecutive_counts_list.append(0)

sum (consecutive_counts_list)

3. Plot the data with breaches symbol is `x`

In [ ]:
fig = px.line(df, x=df.index, y=['VaR_10D', 'rets_10D'], title='NASDAQ100 VaR Back Testing of 99%/10day')
fig.add_scatter(x=df[df['breach'] == 1].index, y=df[df['breach'] == 1]['rets_10D'], mode='markers', marker=dict(color='red', symbol='x'), name='Breach')
fig.update_layout(xaxis_title='Date', yaxis_title='Values')
fig.show()

---

## Question 6

Compute the estimation of variance and then calculate the standard deviation $\sigma$

$$
\sigma_{t+1|t}^{2} = \lambda \sigma^{2}_{t|t-1} + (1-\lambda)r_{t}^{2}
$$

In [ ]:
# Compute returns and squared return
df2 = pd.read_csv("../exam/nasdaq100.csv",parse_dates=["Date"], index_col="Date")
df2['rets'] = np.log(df2['Closing Price']).diff()
df2['sq_rets'] = df2['rets'] **  2

Compute the new VaR using the estimation of EWMA

In [ ]:
# Given lambda
lambda_given = 0.72

# Second day estimation
second_day_variance = np.mean(df2['sq_rets'])

# Create a list of variance estimation
var_est_list = [np.nan, second_day_variance]

# Loop through the data starting from the second index and compute the estimation variance then append the result to the list
for i in range(1, len(df2)-1):
    result = lambda_given * var_est_list[-1] + (1 - lambda_given) * df2.loc[df2.index[i], 'sq_rets']
    var_est_list.append(result)

# Create a new column in the DataFrame to store variance estimation values
df2['vol_est'] = np.sqrt(var_est_list)

# Create a new column for 10D Returns and 10D vol estimation
df2['VaR_est_10D'] = factor * df2['vol_est'] * np.sqrt(10)
df2['rets_10D'] = np.log(df2['Closing Price'].shift(-(11)) / df2['Closing Price'].shift(-1))

# Identify breaches
df2['breach'] = (df2['rets_10D'] < df2['VaR_est_10D']).astype(int)

1. Counting breaches

In [ ]:
mask = df2['breach'] == 1
pct_var_breaches = df2[mask]['breach'].count() / len(df2)
print('No. of Breaches', df2[mask]['breach'].count())
print('Pecentage VaR Breaches', pct_var_breaches)

2. Counting consecutive breaches

In [ ]:
# Convert the breach column to a list for easier manipulation
breach_list = df2['breach'].tolist()

# Initialize variables
consecutive_counts_list = []

# Iterate through the list of breach values
for i in range(len(breach_list) - 1):
    if breach_list[i] == 1 and breach_list[i + 1] == 1:
        consecutive_counts_list.append(1)
    else:
        consecutive_counts_list.append(0)

sum (consecutive_counts_list)

3. Plot the data

In [ ]:
fig = px.line(df2, x=df2.index, y=['VaR_est_10D', 'rets_10D'], title='NASDAQ100 VaR Back Testing EWMA')
fig.add_scatter(x=df2[df2['breach'] == 1].index, y=df2[df2['breach'] == 1]['rets_10D'], mode='markers', marker=dict(color='red', symbol='x'), name='Breach')
fig.update_layout(xaxis_title='Date', yaxis_title='Values')
fig.show()